### Imports

In [2]:
import random
import pandas as pd
from tqdm import tqdm

### Load data files

In [4]:
train_human_labels_trainable_top_10 = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_human_labels_trainable_top_10.csv')
train_human_labels_trainable_top_100 = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_human_labels_trainable_top_100.csv')
print(train_human_labels_trainable_top_10.head())

            ImageID        Source  LabelName  Confidence
0  000002b66c9c498e  verification  /m/01g317           1
1  000002b66c9c498e  verification   /m/09j2d           1
2  000002b66c9c498e  verification   /m/0dzct           1
3  000002b97e5471a0  verification   /m/0cgh4           1
4  000002c707c9895e  verification  /m/01g317           1

### Sort lables frequency

In [6]:
labels_count = train_human_labels_trainable_top_100['LabelName'].value_counts()
print(labels_count.head())

/m/01g317    807090
/m/09j2d     610840
/m/0dzct     331942
/m/07j7r     315026
/m/05s2s     266978
Name: LabelName, dtype: int64

### Merge labels with labels_count

In [8]:
train_human_labels_trainable_top_10 = train_human_labels_trainable_top_10.merge(labels_count.to_frame(), left_on='LabelName', right_index=True, how='left')
train_human_labels_trainable_top_10.rename(columns={'LabelName_x': 'LabelName', 'LabelName_y': 'freq'}, inplace=True)
train_human_labels_trainable_top_10.head()

train_human_labels_trainable_top_100 = train_human_labels_trainable_top_100.merge(labels_count.to_frame(), left_on='LabelName', right_index=True, how='left')
train_human_labels_trainable_top_100.rename(columns={'LabelName_x': 'LabelName', 'LabelName_y': 'freq'}, inplace=True)
train_human_labels_trainable_top_100.head()

,ImageID,Source,LabelName,Confidence,freq,freq
0,000002b66c9c498e,verification,/m/014sv8,1,23974,23974
1,000002b66c9c498e,verification,/m/019nj4,1,31910,31910
2,000002b66c9c498e,verification,/m/01d40f,1,31578,31578
3,000002b66c9c498e,verification,/m/01g317,1,807090,807090
4,000002b66c9c498e,verification,/m/01mzpv,1,25548,25548


### Group labels by image id

In [10]:
top_10_grouped = train_human_labels_trainable_top_10.groupby(['ImageID'])
top_100_grouped = train_human_labels_trainable_top_100.groupby(['ImageID'])

### Split defs

In [7]:
top_10_val_images_count = 10000

### Split top 10

In [8]:
top_10_images = list(top_10_grouped.groups.keys())
print('len(top_10_images) = {}'.format(len(top_10_images)))
random.shuffle(top_10_images)
top_10_images_train = set(top_10_images[:-top_10_val_images_count])
top_10_images_val = set(top_10_images[-top_10_val_images_count:])
print('len(top_10_images_train) = {}'.format(len(top_10_images_train)))
print('len(top_10_images_val) = {}'.format(len(top_10_images_val)))

len(top_10_images) = 1458836

len(top_10_images_train) = 1448836

len(top_10_images_val) = 10000

In [9]:
# Create single label files 

with open(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\lists\top_10_images_train_single.txt', 'w') as f_train,\
    open(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\lists\top_10_images_val_single.txt', 'w') as f_val:
    for name, group in tqdm(top_10_grouped, total=len(top_10_grouped)):
        f = f_val if name in top_10_images_val else f_train
        first_row = group.sort_values(by=['freq'], ascending=False).head(1)
        f.write('{} {}\n'.format(name, first_row['LabelName'].values[0]))

  0%|          | 0/1458836 [00:00<?, ?it/s]

  0%|          | 1/1458836 [00:00<94:05:42,  4.31it/s]

  0%|          | 12/1458836 [00:00<12:22:11, 32.76it/s]

  0%|          | 178/1458836 [00:00<1:14:04, 328.22it/s]

  0%|          | 420/1458836 [00:00<37:12, 653.33it/s]  

...

100%|█████████▉| 1458059/1458836 [10:05<00:00, 2406.50it/s]

100%|█████████▉| 1458299/1458836 [10:05<00:00, 2406.49it/s]

100%|█████████▉| 1458546/1458836 [10:06<00:00, 2406.50it/s]

100%|█████████▉| 1458788/1458836 [10:06<00:00, 2406.50it/s]

100%|██████████| 1458836/1458836 [10:06<00:00, 2406.39it/s]

In [12]:
# Create multi label files 

with open(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\lists\top_10_images_train_multi.txt', 'w') as f_train,\
    open(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\lists\top_10_images_val_multi.txt', 'w') as f_val:
    for name, group in tqdm(top_10_grouped, total=len(top_10_grouped)):
        f = f_val if name in top_10_images_val else f_train
        f.write(name)
        for index, row in group.iterrows():
            f.write(' {}'.format(row['LabelName']))
        f.write('\n')

  0%|          | 0/1458836 [00:00<?, ?it/s]

  0%|          | 1/1458836 [00:00<91:49:16,  4.41it/s]

  0%|          | 332/1458836 [00:00<23:54, 1016.50it/s]

  0%|          | 622/1458836 [00:00<16:42, 1454.63it/s]

  0%|          | 1016/1458836 [00:00<12:37, 1925.68it/s]

...

100%|█████████▉| 1457610/1458836 [06:35<00:00, 3689.47it/s]

100%|█████████▉| 1457981/1458836 [06:35<00:00, 3689.47it/s]

100%|█████████▉| 1458351/1458836 [06:35<00:00, 3689.47it/s]

100%|█████████▉| 1458720/1458836 [06:35<00:00, 3689.46it/s]

100%|██████████| 1458836/1458836 [06:35<00:00, 3689.19it/s]

### Split top 100

In [1]:
top_100_val_images_count = 25000

In [12]:
top_100_images = list(top_100_grouped.groups.keys())
print('len(top_100_images) = {}'.format(len(top_100_images)))
random.shuffle(top_100_images)
top_100_images_train = set(top_100_images[:-top_100_val_images_count])
top_100_images_val = set(top_100_images[-top_100_val_images_count:])
print('len(top_100_images_train) = {}'.format(len(top_100_images_train)))
print('len(top_100_images_val) = {}'.format(len(top_100_images_val)))


  0%|          | 0/1663527 [00:00<?, ?it/s]

  0%|          | 1/1663527 [00:00<197:29:48,  2.34it/s]

  0%|          | 36/1663527 [00:00<7:40:32, 60.20it/s] 

  0%|          | 236/1663527 [00:00<1:22:04, 337.76it/s]

  0%|          | 444/1663527 [00:00<49:56, 555.07it/s]  

...

100%|█████████▉| 1662486/1663527 [09:58<00:00, 2778.68it/s]

100%|█████████▉| 1662779/1663527 [09:58<00:00, 2778.66it/s]

100%|█████████▉| 1663068/1663527 [09:58<00:00, 2778.68it/s]

100%|█████████▉| 1663354/1663527 [09:58<00:00, 2778.69it/s]

100%|██████████| 1663527/1663527 [09:58<00:00, 2778.55it/s]

In [13]:
# Create multi label files 

with open(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\lists\top_100_images_train_multi.txt', 'w') as f_train,\
    open(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\lists\top_100_images_val_multi.txt', 'w') as f_val:
    for name, group in tqdm(top_100_grouped, total=len(top_100_grouped)):
        f = f_val if name in top_100_images_val else f_train
        f.write(name)
        for index, row in group.iterrows():
            f.write(' {}'.format(row['LabelName']))
        f.write('\n')


  0%|          | 0/1663527 [00:00<?, ?it/s]

  0%|          | 1/1663527 [00:00<197:29:48,  2.34it/s]

  0%|          | 36/1663527 [00:00<7:40:32, 60.20it/s] 

  0%|          | 236/1663527 [00:00<1:22:04, 337.76it/s]

  0%|          | 444/1663527 [00:00<49:56, 555.07it/s]  

...

100%|█████████▉| 1662486/1663527 [09:58<00:00, 2778.68it/s]

100%|█████████▉| 1662779/1663527 [09:58<00:00, 2778.66it/s]

100%|█████████▉| 1663068/1663527 [09:58<00:00, 2778.68it/s]

100%|█████████▉| 1663354/1663527 [09:58<00:00, 2778.69it/s]

100%|██████████| 1663527/1663527 [09:58<00:00, 2778.55it/s]